## Oscilateur harmonique libre et amorti (ou non)

Nous considérons l'oscilateur harmonique libre et amorti.

Une masse $m$ est accrochée à un ressort de constante de raideur $k$ et écartée de sa position d'équilibre de $x_0$, puis lâchée avec une vitesse initiale $v_0$.

La masse est plongée dans un fluide et le coefficient de frottement visquex laminaire est $b_l= K\eta$

$$\Omega_0=\sqrt{k/m}$$

$$T_0=2\pi/\Omega_0$$

$$\gamma=(b_l/2m)$$

L'équation différentielle du mouvement est:

$$\ddot x (t) + 2 \gamma \dot x(t) +\Omega_0^2 x(t) = 0$$

l'équation caractéristique associée est

$$\lambda^2+2\gamma \lambda+\Omega_0^2=0$$

son déterminant réduit est $\Delta '=(\gamma^2-\Omega_0^2)$

Si $\Delta ' \neq 0$, les racines de l'équation sont:

$$\lambda_1=-\gamma +(\Delta ')^{0.5}$$

$$\lambda_2=-\gamma -(\Delta ')^{0.5}$$

Les solutions de l'équadiff sont alors de la forme

$$x_c(t)=C_1 e^{\lambda_1 t}+C_2 e^{\lambda_2 t}$$

Les conditions initiales donnent 
$$C_1=\frac{\lambda_2 x_0 - v_0}{\lambda_2-\lambda_1}$$

$$C_2=\frac{\lambda_1 x_0 - v_0}{\lambda_1-\lambda_2}$$

Attention! $\lambda_1$ et $\lambda_2$, $C_1$ et $C_2$ sont complexes

La solution physique est la partie réelle de $x_c$.

Si $\Delta ' = 0$, la solution est la partie réelle de:

$$x_c(t)=[x_0+(v_0+\gamma x_0)t ]e^{-\gamma t}$$



import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


In [ ]:
import numpy as np
from cmath import *

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider
from bokeh.plotting import figure, show, ColumnDataSource

from bokeh.io import push_notebook, show, output_notebook
from ipywidgets import interact
#from bokeh.resources import INLINE 

output_notebook()

In [ ]:
def omega0(k,m):
    oz=np.sqrt(k/m)
    return oz

def gamma(bl,m):
    ga=bl/(2*m)
    return ga

def deltaprime(k,m,bl):
    dp=gamma(bl,m)**2-omega0(k,m)**2+0j
    return dp

def osciamo(k,m,bl,xzero,vzero,t):
    if (deltaprime(k,m,bl)!=0):
        gamma1=-gamma(bl,m)+deltaprime(k,m,bl)**(1/2)
        gamma2=-gamma(bl,m)-deltaprime(k,m,bl)**(1/2)
        C1=(gamma2*xzero-vzero)/(gamma2-gamma1)
        C2=(gamma1*xzero-vzero)/(gamma1-gamma2)
        solc=C1*np.exp(gamma1*t)+C2*np.exp(gamma2*t)
        sol=solc.real
    else:
        sol=(xzero+(vzero+gamma(bl,m)*xzero)*t)*np.exp(-gamma(bl,m)*t)
    return sol    
    

In [ ]:
p = figure(plot_width=800, plot_height=400)

#valeurs initiales
k=50
m=5
bl=0
xzero=1
vzero=0

t = np.linspace(0,40,1000)
x1 = osciamo(k,m,bl,xzero,vzero,t)

pos = p.line(t,x1,line_width=3,legend="position")

p.legend.location = "top_right"
p.legend.click_policy="hide"


def update(m=5, k = 50, bl = 0): #, xzero= 1 , vzero = 0):
    pos.data_source.data['y'] = osciamo(k,m,bl,xzero,vzero,t)
    push_notebook()
    print('Omega_0 = {:0.2f}[rad/s]'.format(omega0(k,m)),",  gamma = ", gamma(bl,m)) 
    dp =  deltaprime(k,m,bl)**0.5
    dpr = dp.real
    dpi = dp.imag
    print('-')
    print('Partie réelle de racine de delta prime {:0.2f}[rad/s]'.format(dpr))
    print('-')
    print('Partie imaginaire de racine de delta prime {:0.2f}[rad/s]'.format(dpi))

show(p, notebook_handle=True)

interact(update, m = (1,10,1), k = (1,50,1), bl = (0,200,1));

## Oscilateur forcé. 

On conserve les mêmes notations, mais cette fois on soumet l'oscilateur à une force $F_0 \cos (\omega_e t)$

Comme la force d'excitation est obtenue par le déplacement sinusoidal d'une membrane, avec un déplacement d'amplitude $x_{e,0}$, qui provoque sur le ressort une force d'amplitude $F_0=k x_{e,0}$, le déplacement de l'excitation est égal à

$$\frac{f}{k}\cos(\omega_e t)$$

L'équadiff devient 

$$\ddot x (t) + 2 \gamma \dot x(t) +\Omega_0^2 x(t) = (F_0/m) \cos (\omega_e t)$$

En notation complexe et avec $f=F_0/m$

$$\ddot x (t) + 2 \gamma \dot x(t) +\Omega_0^2 x(t) = f e^{i\omega_e t}$$

La solution en régime permanent est $x_{1,c}(t)= \chi_0 e^{i\omega_e t}$ avec $\chi_0=A e^{i \varphi}$. $A(\omega_e)$ est l'amplitude et $\varphi(\omega_e)$ le déphasage.

$$\chi_0 = \frac{f}{-(\omega_e^2-\Omega_0^2)+i(2\gamma\omega_e)}$$

La solution générale est la somme du régime permanent  plus la solution générale de l'équation différentielle sans second membre trouvée au paragraphe précédent ($x_{2,c}(t)$).

Les constantes d'intégration $C_1$ et $C_2$ se déterminent sur $x_{c}(t)=x_{1,c}(t)+x_{2,c}(t)$

In [ ]:
omegae_v = np.linspace(5,15,10000)

#amplitude de l'excitation
def exci(omegae,k,f,t):
    ex=(f/k)*np.cos(omegae*t)
    return ex

def chizero(omegae,k,m,bl,f):
    chizero=f/complex((omega0(k,m)**2-omegae**2),(2*gamma(bl,m)*omegae))
    return chizero

def amplitudes(k,m,bl,f):
    amp=[]
    for i in omegae_v:
        amp.append(abs(chizero(i,k,m,bl,f)))
    return amp

def dephasages(k,m,bl,f):
    deph=[]
    for i in omegae_v:
        deph.append(phase(chizero(i,k,m,bl,f)))
    return deph

#x1(t) complexe
def regpermc(omegae,k,m,bl,f,t):
    rp=chizero(omegae,k,m,bl,f)*complex(np.cos(omegae*t),np.sin(omegae*t))
    return rp

#x2(t) complexe, avec C1 et C2 déterminés grace à x0 et v0
def xdeuxdetc(omegae,k,m,bl,f,xzero,vzero,t):
    if (deltaprime(k,m,bl)!=0):
        gamma1=-gamma(bl,m)+deltaprime(k,m,bl)**(1/2)
        gamma2=-gamma(bl,m)-deltaprime(k,m,bl)**(1/2)
        C1=(gamma2*xzero-vzero-gamma2*chizero(omegae,k,m,bl,f)+(0+1j)*chizero(omegae,k,m,bl,f)*omegae)/(gamma2-gamma1)
        C2=(gamma1*xzero-vzero-gamma1*chizero(omegae,k,m,bl,f)+(0+1j)*chizero(omegae,k,m,bl,f)*omegae)/(gamma1-gamma2)
        sol=C1*np.exp(gamma1*t)+C2*np.exp(gamma2*t)
    else:
        A=xzero-chizero(omegae,k,m,bl,f)
        B=vzero+gamma(bl,m)*xzero-gamma(bl,m)*chizero(omegae,k,m,bl,f)-(0+1j)*chizero(omegae,k,m,bl,f)*omegae
        sol=(A+B*t)*np.exp(-gamma(bl,m)*t)
    return sol    
    


In [ ]:
p2 = figure(plot_width=950, plot_height=400,y_range=(-1, 1))

#valeurs initiales (pour l'oscilateur dans l'air)
# air : k=15, m=0.19, bl=0.02, f=1, xzero=0, vzero=0
# eau : 
air = (15,0.19,0.02,1,0,0)
eau = (15,0.19,0.3,1,0,0)
choix = air
k = choix[0]
m = choix[1]
bl = choix[2]
f = choix[3]
xzero = choix[4]
vzero = choix[5]
omegae = 7.1 #Pulsation d'excitation
tmin = 0
tmax = 1000

t2 = np.linspace(0,100,5000)

def permreal(omegae,k,m,bl,f,t):
    pr=[]
    for i in t:
        pr.append(regpermc(omegae,k,m,bl,f,i).real)
    pr=np.array(pr)
    return pr

def x2real(omegae,k,m,bl,f,xzero,vzero,t):
    x2r=[]
    for i in t:
        x2r.append(xdeuxdetc(omegae,k,m,bl,f,xzero,vzero,i).real)
    x2r=np.array(x2r)
    return x2r


excitation = p2.line(t2,exci(omegae,k,f,t2),line_width=3, legend="excitation")
permanent = p2.line(t2,permreal(omegae,k,m,bl,f,t2),color="orange",line_width=3,legend="régime permanent")
amorti = p2.line(t2,x2real(omegae,k,m,bl,f,xzero,vzero,t2),color="green",line_width=3,legend="solution particulière")
total = p2.line(t2,permreal(omegae,k,m,bl,f,t2)+x2real(omegae,k,m,bl,f,xzero,vzero,t2),color="red",line_width=3,legend="solution totale")

p2.legend.location = "top_right"
p2.legend.click_policy="hide"


dp = figure(plot_width=400, plot_height=300)
deph=dp.line(omegae_v,dephasages(k,m,bl,f),line_width=3)

# PROBLEM LINES
deph_vx=np.array([omegae])
deph_vy=np.array([phase(chizero(omegae,k,m,bl,f))])
deph_v=dp.circle(deph_vx,deph_vy,color="red",size=8)

ap = figure(plot_width=400, plot_height=300)
amp=ap.line(omegae_v,amplitudes(k,m,bl,f),line_width=3)

def update2(milieu, omegae=8.3): #, xzero= 1 , vzero = 0):
    if milieu=="eau": choix = eau
    if milieu=="air": choix=air
    k = choix[0]
    m = choix[1]
    bl = choix[2]
    f = choix[3]
    xzero = choix[4]
    vzero = choix[5]
    excitation.data_source.data['y'] = exci(omegae,k,f,t2)
    permanent.data_source.data['y'] = permreal(omegae,k,m,bl,f,t2)
    amorti.data_source.data['y'] = x2real(omegae,k,m,bl,f,xzero,vzero,t2)
    total.data_source.data['y'] = permreal(omegae,k,m,bl,f,t2)+x2real(omegae,k,m,bl,f,xzero,vzero,t2)
    deph.data_source.data['y'] = dephasages(k,m,bl,f)
    amp.data_source.data['y'] = amplitudes(k,m,bl,f)
#PROBLEM LINES
    deph_v.data_source.data['x'] = np.array([omegae])
    deph_v.data_source.data['y'] = np.array([phase(chizero(omegae,k,m,bl,f))])

    push_notebook()

show(column(p2,row(dp,ap)), notebook_handle=True)

interact(update2, omegae = (5,15,.1),milieu=["eau","air"]);